In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import Row
from pyspark.sql.functions import col,pandas_udf, PandasUDFType,count
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
import nltk

#Install Tagged file from databricks table
multiban = spark.table("allban")
multiban.cache()

Out[ 1 ]: DataFrame[label: int, Text: string]

In [2]:
multiban=multiban.na.drop()
display(multiban.groupby('label').count())

label,count
1,6367
0,29281


In [3]:
from pyspark.sql.types import IntegerType
multiban = multiban.withColumn("label", multiban["label"].cast(IntegerType()))

In [4]:
train_df, test_df = multiban.randomSplit([0.65, 0.35], seed = 2018)

print("Training Dataset Count: " + str(train_df.count()))
print("Test Dataset Count: " + str(test_df.count()))

Training Dataset Count: 23092
Test Dataset Count: 12556

In [5]:
train_df.cache()
test_df.cache()

Out[ 9 ]: DataFrame[label: int, Text: string]

In [6]:
from pyspark.ml.feature import HashingTF, IDF, RegexTokenizer,StopWordsRemover,VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression,NaiveBayes,LinearSVC,GBTClassifier
from mleap.sklearn.preprocessing.data import FeatureExtractor, LabelEncoder, ReshapeArrayToN1
from pyspark.ml.evaluation import RegressionEvaluator,MulticlassClassificationEvaluator,BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit,CrossValidator
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import *

In [7]:
print(StopWordsRemover.loadDefaultStopWords("turkish"))

['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani']

In [8]:
stopWordstr = ['❤️', '◻', '😑', '🙏🏻', '🌸', '🙌🏻',  '😇',  '😊', '😢', '⭐️', '🌼', '🙈', '🍀', '💗', '💕', '😊', '🌺', '😅', '💵', '🙈', '🙏🙏🙏', '🙏🙏', '✔️', '😌🙏🏻', '😍', '👍', '👍👍', '👍👍👍', ':)', '👏🏼', '👏🏼👏🏼', '👏🏼👏🏼👏🏼', '🤔', '☺️', '😑😑', '😑', ':(', '😊', '👌👌', '👌', '💃🏻', '✌🏻', ':))', ':)))', '🎃','😉','😄','🤗']

regexTokenizer = RegexTokenizer(inputCol="Text", outputCol="words", pattern=' |,|;|-|_|\*|\t|\!|\.|\*|\:|\(|\|\"|\&|\$|\|\#|\}|\]|\[|\)|\{|\/|\'|<|>',toLowercase=True)

remover = StopWordsRemover(inputCol="words", outputCol="filtered",stopWords =stopWordstr)

hashtf = HashingTF(inputCol="filtered", outputCol='tf')

idf = IDF(inputCol='tf', outputCol="tffeatures")

va = VectorAssembler(inputCols=["tf", "tffeatures"], outputCol="features") 

lr = LogisticRegression()
nv= NaiveBayes()
lsvc=LinearSVC()
gbt = GBTClassifier()

pipelinelr = Pipeline(stages=[regexTokenizer,remover,hashtf, idf,va,lr])
pipelinenv = Pipeline(stages=[regexTokenizer,remover,hashtf, idf, nv])
pipelinelsvc = Pipeline(stages=[regexTokenizer,remover,hashtf, idf, lsvc])
pipelinegbt = Pipeline(stages=[regexTokenizer,remover,hashtf, idf, gbt])


In [9]:

paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.0001,0.006,0.003,0.01,0.03])
             .addGrid(idf.minDocFreq,[2,3,4])
             .addGrid(hashtf.numFeatures, [2**3,2**18])
             .addGrid(hashtf.binary, [True,False])
             .addGrid(lr.fitIntercept, [True,False])
             .addGrid(lr.standardization, [True,False])
             .addGrid(lr.elasticNetParam, [0.01,0.05,0.1])
             .addGrid(lr.aggregationDepth, [2,3])
             .addGrid(lr.maxIter,[5,1000])
             .addGrid(lr.family,['binomial'])
             .addGrid(lr.tol,[1e-06,1e-01])
             .build()  )

cvlr = CrossValidator(estimator=pipelinelr, evaluator=MulticlassClassificationEvaluator(), estimatorParamMaps=paramGrid)

cvModel = cvlr.fit(train_df)
modellr = cvModel.bestModel

In [10]:
modellr.stages[4].extractParamMap()

In [11]:
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.006])
             #.addGrid(idf.minDocFreq,[2])
             .addGrid(hashtf.numFeatures, [2**18])
             #.addGrid(hashtf.binary, [True])
             #.addGrid(lr.fitIntercept, [True])
             #.addGrid(lr.standardization, [True])
             #.addGrid(lr.elasticNetParam, [0.1])
             #.addGrid(lr.aggregationDepth, [2])
             #.addGrid(lr.maxIter,[1])
             #.addGrid(lr.tol,[1e-06])
             .build()  )

cvlr = CrossValidator(estimator=pipelinelr, evaluator=MulticlassClassificationEvaluator(), estimatorParamMaps=paramGrid)

cvModel = cvlr.fit(train_df)
modellr = cvModel.bestModel

In [12]:
predictions = modellr.transform(test_df)
 

predictions = predictions.select(col("label").cast("Float"),col("prediction"))
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

 
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % accuracy)
 
evaluatorf1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1 = evaluatorf1.evaluate(predictions)
print("f1 = %g" % f1)
 
evaluatorwp = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
wp = evaluatorwp.evaluate(predictions)
print("weightedPrecision = %g" % wp)
 
evaluatorwr = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
wr = evaluatorwr.evaluate(predictions)
print("weightedRecall = %g" % wr)

evaluatorpr = BinaryClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="areaUnderPR")
pr = evaluatorpr.evaluate(predictions)
print("areaUnderPR = %g" % pr)



--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-837974679672910> in <module> () 
 24 print ( "weightedRecall = %g" % wr ) 
 25 
 ---> 26 evaluatorpr = BinaryClassificationEvaluator ( labelCol = "label" , predictionCol = "prediction" , metricName = "areaUnderPR" ) 
 27 pr = evaluatorpr . evaluate ( predictions ) 
 28 print ( "areaUnderPR = %g" % pr ) 

 /databricks/spark/python/pyspark/__init__.py in wrapper (self, *args, **kwargs) 
 103 raise TypeError ( "Method %s forces keyword arguments." % func . __name__ ) 
 104 self . _input_kwargs = kwargs 
 --> 105 return func ( self , ** kwargs ) 
 106 return wrapper 
 107 

 TypeError : __init__() got an unexpected keyword argument 'predictionCol'

In [13]:
evaluatorpr = BinaryClassificationEvaluator(labelCol="label", predictionCol="rawPrediction", metricName="areaUnderPR")
pr = evaluatorpr.evaluate(predictions)
print("areaUnderPR = %g" % pr)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-348649863844189> in <module> () 
 ----> 1 evaluatorpr = BinaryClassificationEvaluator ( labelCol = "label" , predictionCol = "rawPrediction" , metricName = "areaUnderPR" ) 
 2 pr = evaluatorpr . evaluate ( predictions ) 
 3 print ( "areaUnderPR = %g" % pr ) 

 /databricks/spark/python/pyspark/__init__.py in wrapper (self, *args, **kwargs) 
 103 raise TypeError ( "Method %s forces keyword arguments." % func . __name__ ) 
 104 self . _input_kwargs = kwargs 
 --> 105 return func ( self , ** kwargs ) 
 106 return wrapper 
 107 

 TypeError : __init__() got an unexpected keyword argument 'predictionCol'

In [14]:
paramGrid = (ParamGridBuilder()
             .addGrid(hashtf.numFeatures, [2**16])
             #.addGrid(lsvc.regParam, [0.1])
             .build()  )

cvlsvc = CrossValidator(estimator=pipelinelsvc, evaluator=MulticlassClassificationEvaluator(), estimatorParamMaps=paramGrid)

cvModel = cvlsvc.fit(train_df)
modellsvc = cvModel.bestModel



In [15]:
modellsvc.stages[5].extractParamMap()

Out[ 42 ]: 
{Param(parent='LinearSVC_46a5ae11c6867ccf8ef7', name='fitIntercept', doc='whether to fit an intercept term'): True,
 Param(parent='LinearSVC_46a5ae11c6867ccf8ef7', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent='LinearSVC_46a5ae11c6867ccf8ef7', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0)'): 1e-06,
 Param(parent='LinearSVC_46a5ae11c6867ccf8ef7', name='threshold', doc='threshold in binary classification prediction applied to rawPrediction'): 0.0,
 Param(parent='LinearSVC_46a5ae11c6867ccf8ef7', name='standardization', doc='whether to standardize the training features before fitting the model'): True,
 Param(parent='LinearSVC_46a5ae11c6867ccf8ef7', name='regParam', doc='regularization parameter (>= 0)'): 0.1,
 Param(parent='LinearSVC_46a5ae11c6867ccf8ef7', name='maxIter', doc='maximum number of iterations (>= 0)'): 100,
 Param(parent='LinearSVC_46a5ae11c6867ccf8ef7', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'): 2,
 Param(parent='LinearSVC_46a5ae11c6867ccf8ef7', name='labelCol', doc='label column name'): 'label',
 Param(parent='LinearSVC_46a5ae11c6867ccf8ef7', name='featuresCol', doc='features column name'): 'features',
 Param(parent='LinearSVC_46a5ae11c6867ccf8ef7', name='rawPredictionCol', doc='raw prediction (a.k.a. confidence) column name'): 'rawPrediction'}

In [16]:

paramGrid = (ParamGridBuilder()
             .addGrid(hashtf.numFeatures, [2**16])
             .addGrid(nv.smoothing, [0.065])
             .build()  )

cvnv = CrossValidator(estimator=pipelinenv, evaluator=MulticlassClassificationEvaluator(), estimatorParamMaps=paramGrid)

cvModel = cvnv.fit(train_df)
modelnv = cvModel.bestModel



predictions = modelnv.transform(test_df)
 

predictions = predictions.select(col("label").cast("Float"),col("prediction"))
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

 
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % accuracy)
 
evaluatorf1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1 = evaluatorf1.evaluate(predictions)
print("f1 = %g" % f1)
 
evaluatorwp = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
wp = evaluatorwp.evaluate(predictions)
print("weightedPrecision = %g" % wp)
 
evaluatorwr = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
wr = evaluatorwr.evaluate(predictions)
print("weightedRecall = %g" % wr)

In [17]:
predictions = modellsvc.transform(test_df)
 

predictions = predictions.select(col("label").cast("Float"),col("prediction"))
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

 
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % accuracy)
 
evaluatorf1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1 = evaluatorf1.evaluate(predictions)
print("f1 = %g" % f1)
 
evaluatorwp = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
wp = evaluatorwp.evaluate(predictions)
print("weightedPrecision = %g" % wp)
 
evaluatorwr = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
wr = evaluatorwr.evaluate(predictions)
print("weightedRecall = %g" % wr)

Test Error = 0.0833864
Accuracy = 0.916614
f1 = 0.914963
weightedPrecision = 0.914109
weightedRecall = 0.916614

In [18]:
paramGrid = (ParamGridBuilder()
             .addGrid(hashtf.numFeatures, [2**12])
             .build()  )

cvgbt = CrossValidator(estimator=pipelinegbt, evaluator=MulticlassClassificationEvaluator(), estimatorParamMaps=paramGrid)

cvModel = cvgbt.fit(train_df)
modelgbt = cvModel.bestModel


In [19]:
predictions = modelgbt.transform(test_df)
 

predictions = predictions.select(col("label").cast("Float"),col("prediction"))
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

 
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % accuracy)
 
evaluatorf1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1 = evaluatorf1.evaluate(predictions)
print("f1 = %g" % f1)
 
evaluatorwp = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
wp = evaluatorwp.evaluate(predictions)
print("weightedPrecision = %g" % wp)
 
evaluatorwr = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
wr = evaluatorwr.evaluate(predictions)
print("weightedRecall = %g" % wr)

Test Error = 0.128616
Accuracy = 0.871384
f1 = 0.845814
weightedPrecision = 0.873945
weightedRecall = 0.871384

In [20]:
display(predictionslr,5)

In [21]:
%sh 
rm -rf /tmp/model_export
mkdir /tmp/model_export

In [22]:
import sys
sys.path.append('/opt/libs/mleap/python')

import mleap.pyspark
from mleap.pyspark.spark_support import SimpleSparkSerializer

        
modellr.serializeToBundle("jar:file:/tmp/model_export/LR_model_vsX-json.zip",predictionslr)

In [23]:
predictionAndLabels = test_df.map(lambda lp: (float(modellr.predict(lp.features)), lp.label))

# Instantiate metrics object
metrics = BinaryClassificationMetrics(predictionAndLabels)

# Area under precision-recall curve
print("Area under PR = %s" % metrics.areaUnderPR)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2686753707922364> in <module> () 
 ----> 1 predictionAndLabels = test_df . map ( lambda lp : ( float ( modellr . predict ( lp . features ) ) , lp . label ) ) 
 2 
 3 # Instantiate metrics object 
 4 metrics = BinaryClassificationMetrics ( predictionAndLabels ) 
 5 

 /databricks/spark/python/pyspark/sql/dataframe.py in __getattr__ (self, name) 
 1243 if name not in self . columns : 
 1244 raise AttributeError(
 -> 1245 "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
 1246 jc = self . _jdf . apply ( name ) 
 1247 return Column ( jc ) 

 AttributeError : 'DataFrame' object has no attribute 'map'